In [2]:
import numpy as np
import pandas as pd
import datetime as dt

import json
from PIL import Image
import urllib.request
import requests
from bs4 import BeautifulSoup

# Background

https://www.irishtimes.com/life-and-style/people/ireland-s-darkest-oddest-and-weirdest-secrets-uncovered-1.3418059

https://www.duchas.ie/download/17.01.26-irish-folklore-and-tradition.pdf

https://www.frontiersin.org/articles/10.3389/fphar.2020.584595/full

# Example
https://www.duchas.ie/en/tpc/cbes/5098227?p=1

# Full pipeline

In [9]:
pp = 1
base_url = 'https://www.duchas.ie'
all_topics = base_url+'/en/tpc/cbes'
print(all_topics)
page_all = requests.get(all_topics)
soup_all = BeautifulSoup(page_all.content, 'html.parser')
all_topics_list = soup_all.find_all('div',{'class':'topics'})[0].find_all('a',href=True)

https://www.duchas.ie/en/tpc/cbes
[<a class="svelte-5bilvy" href="/en/cbes/stories?TopicID=5098227"><span class="svelte-5bilvy">agriculture (~2,659)</span> </a>, <a class="svelte-5bilvy" href="/en/cbes/stories?TopicID=5287781"><span class="svelte-5bilvy">animal husbandry (~2,587)</span> </a>, <a class="svelte-5bilvy" href="/en/cbes/stories?TopicID=5287782"><span class="svelte-5bilvy">calling animals (~71)</span> </a>, <a class="svelte-5bilvy" href="/en/cbes/stories?TopicID=5287779"><span class="svelte-5bilvy">medicine for animal sicknesses (~175)</span> </a>, <a class="svelte-5bilvy" href="/en/cbes/stories?TopicID=5287774"><span class="svelte-5bilvy">blackleg (~21)</span> </a>, <a class="svelte-5bilvy" href="/en/cbes/stories?TopicID=5287780"><span class="svelte-5bilvy">elf-shot (~18)</span> </a>, <a class="svelte-5bilvy" href="/en/cbes/stories?TopicID=5287773"><span class="svelte-5bilvy">farcy (~12)</span> </a>, <a class="svelte-5bilvy" href="/en/cbes/stories?TopicID=5287778"><span cla

In [51]:
all_data = {}
t_ix = 0
curr_idx = 0

In [76]:
curr_idx = curr_idx + t_ix + 1
for t_ix, t_url in enumerate(all_topics_list[curr_idx:curr_idx+40]):
    topic_url = t_url['href']

    topic_url_i = base_url+topic_url
    page_i = requests.get(topic_url_i)
    soup_i = BeautifulSoup(page_i.content, 'html.parser')
    topic_name_i = soup_i.find_all('div',{'class':'pageHeader'})[0].text.replace('\n','')
    n_results_topic_i = int(soup_i.find('div',{'class':'resultCount'}).text.split('count: '
                                            )[1].replace(' ','').replace(',','').split('\r')[0])
    n_pages_topic_i = int(soup_i.find('span',{'class':'lastPageNum'}).text.replace(' ','').replace('/',''))
    print(dt.datetime.now(),'\tStarting',topic_name_i)
    out_topic_dict = {}

#     for pp in range(1,14):
    for pp in range(1,n_pages_topic_i+1):
        topic_url_pp = base_url+topic_url+'?p=%i'%pp
        page_pp = requests.get(topic_url_pp)
        soup_pp = BeautifulSoup(page_pp.content, 'html.parser')
        try:
            all_stories_pp = soup_pp.find_all('section',
                                        {'class':'contentListBox'})[0].find_all('div',
                                                                                {'class':'contentDesc'})
        except:
            print("\t\tbroken link:",topic_url_pp)
            continue

        print('\t',topic_name_i,'page: %03i/%i ... %i results on this page'%(pp,n_pages_topic_i,
                                                        len(all_stories_pp)), dt.datetime.now())
#         for sub_story_i in all_stories_pp[:2]:
        for sub_story_i in all_stories_pp:
            sub_story_url = base_url+sub_story_i.find('a',href=True)['href']
            page_i = requests.get(sub_story_url)
            soup_i = BeautifulSoup(page_i.content, 'html.parser')
            text_i = soup_i.find_all('div',{'class':'transcript'})
            meta_data_list = [dict(zip([i.text for i in x.find_all('dt')],
                                       [i.text for i in x.find_all('dd')]))
                              for x in soup_i.find_all('dl')]

            raw_text_i = "\n\n".join([i.text for i in
                                      soup_i.find_all('div',{'class':'transcript'})])
            try:
                bibl1 = soup_i.find_all('div',{'id':'itemBibli'})[0].find('h2').text
                bibl2 = soup_i.find_all('div',{'id':'itemBibli'})[0].find('p').text

            except:
                print("\t\tbroken link:",topic_url_pp)
                continue

            school_info_i = soup_i.find_all('div',{'class':'xml'})[0].text
            page_info_i = soup_i.find_all('div',{'class':'xml'})[1].text
            title_i = soup_i.find_all('div',{'class':'xml'}
                                     )[2].text.replace('“','').replace('”','')
            label_id = sub_story_url.replace('https://www.duchas.ie/en/cbes/','')
            out_topic_dict[label_id] = {'title':title_i,
                                        'bib':[bibl1,bibl2,page_info_i],
                                        'url':sub_story_url,
                                        'text':raw_text_i,
                                        'school_info':school_info_i,
                                        'metadata':meta_data_list
                                         }

    with open('../data/out/schools-collection_%s.json'%topic_name_i, 'w') as fp:
        json.dump(out_topic_dict, fp)

    print(dt.datetime.now(),'\tSaving....',topic_name_i,t_ix,'\n')

    all_data[topic_name_i] = out_topic_dict

Agriculture
2705
2023-10-19 12:27:44.954105 	Starting Agriculture
Animal
2554
2023-10-19 12:27:45.470817 	Starting Animal
Calling
71
2023-10-19 12:27:45.974413 	Starting Calling
Medicine
173
2023-10-19 12:27:46.595149 	Starting Medicine
Blackleg
21
2023-10-19 12:27:47.067800 	Starting Blackleg
Elf-shot
18
2023-10-19 12:27:47.561587 	Starting Elf-shot
Farcy
12
2023-10-19 12:27:48.044474 	Starting Farcy
Worms
35
2023-10-19 12:27:48.551667 	Starting Worms
Butter
3324
2023-10-19 12:27:49.063880 	Starting Butter
Potatoes
2723
2023-10-19 12:27:49.567587 	Starting Potatoes
Fishing
214
2023-10-19 12:27:50.060458 	Starting Fishing
Buying
3585
2023-10-19 12:27:50.592139 	Starting Buying
Trades
4749
2023-10-19 12:27:51.127820 	Starting Trades
Basket-making
478
2023-10-19 12:27:51.680090 	Starting Basket-making
Candle-making
735
2023-10-19 12:27:52.192019 	Starting Candle-making
Coopering
88
2023-10-19 12:27:52.705338 	Starting Coopering
Dyeing
133
2023-10-19 12:27:53.250242 	Starting Dyeing
Lime-

2023-10-19 14:33:38.421804 	Starting Agriculture
	 Agriculture page: 001/136 ... 20 results on this page 2023-10-19 14:33:39.080325
[{'Location:': 'Eoghanacht, Co. Galway '}, {'Topics': 'genre poetry folk poetry (~9,504)\n\t\t\t\t     activities economic activities agriculture (~2,659)\n\t\t\t\t     products food products (~3,601)\n\t\t\t\t    ', 'Language': 'Irish '}, {'Content and data:': '© 2023 National Folklore Collection, UCD ', 'Technical solution:': '© 2023 Gaois, Fiontar & Scoil na Gaeilge, DCU\n\t\t\t\t', 'Contact:': 'eolas@duchas.ie', 'Social media:': ' Facebook  Twitter'}]
		broken link: https://www.duchas.ie/en/cbes/stories?TopicID=5098227&Page=1&PerPage=20
[{'Location:': 'Cill Rónáin, Co. Galway '}, {'Topics': 'activities economic activities agriculture (~2,659)\n\t\t\t\t     events events (by time of year) (~11,476)\n\t\t\t\t    ', 'Language': 'Irish '}, {'Collector': 'Pádhraic Ó Maoláin  ', 'Gender': 'Male', 'Address': 'Cill Rónáin, Co. Galway '}, {'Informant': 'Bairtle

KeyboardInterrupt: 

_________________________